In [54]:
using LinearAlgebra, Random, Statistics

In [72]:
"""
    tridiag(M::Tridiagonal{T,<:Array}, f::Vector{T})::Vector{T} where T
Solve the tridiagonal system of linear equations described by the tridiagonal
matrix `M` with right-hand-side `g` assuming one of the eigenvalues is zero
(which results in a singular matrix so the general Thomas algorithm has been
modified slightly).
Reference CPU implementation per Numerical Recipes, Press et. al 1992 (§ 2.4)
"""
function tridiag(M::Tridiagonal{T,<:Array}, f::Vector{T})::Vector{T} where T
    N = length(f)
    ϕ = similar(f)
    γ = similar(f)

    β    = M.d[1]
    ϕ[1] = f[1] / β

    for j = 2:N
        γ[j] = M.du[j-1] / β
        β    = M.d[j] - M.dl[j-1] * γ[j]

        # This should only happen on last element of forward pass for problems
        # with zero eigenvalue. In that case the algorithmn is still stable.
        abs(β) < 1.0e-12 && break

        ϕ[j] = (f[j] - M.dl[j-1] * ϕ[j-1]) / β
    end

    for j = 1:N-1
        k = N-j
        ϕ[k] = ϕ[k] - γ[k+1] * ϕ[k+1]
    end

    return ϕ
end

tridiag

In [9]:
zF = [1, 2, 4, 7, 11]

3-element Array{Float64,1}:
 1.5
 2.5
 3.5

In [34]:
function grid(zF)
    Nz = length(zF) - 1
    ΔzF = [zF[k+1] - zF[k] for k in 1:Nz]
    zC = [(zF[k] + zF[k+1]) / 2 for k in 1:Nz]
    ΔzC = [zC[k+1] - zC[k] for k in 1:Nz-1]
    return zF, zC, ΔzF, ΔzC
end

grid (generic function with 1 method)

In [146]:
zF, zC, ΔzF, ΔzC = grid([1, 2, 4, 7, 11, 16, 22, 29, 37])

([1, 2, 4, 7, 11, 16, 22, 29, 37], [1.5, 3.0, 5.5, 9.0, 13.5, 19.0, 25.5, 33.0], [1, 2, 3, 4, 5, 6, 7, 8], [1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])

In [140]:
@show Nz = length(zF) - 1
ld = [1/ΔzF[k] for k in 1:Nz-1]
ud .= ld

Nz = length(zF) - 1 = 8


7-element Array{Float64,1}:
 1.0                
 0.5                
 0.3333333333333333 
 0.25               
 0.2                
 0.16666666666666666
 0.14285714285714285

In [142]:
δ(k) = 1/ΔzC[k-1] - 1/ΔzC[k]

δ (generic function with 1 method)

In [144]:
d = [-1/ΔzC[1], [-δ(k) for k in 2:Nz-1]..., -1/ΔzC[Nz-1]]

8-element Array{Float64,1}:
 -0.6666666666666666  
 -0.2666666666666666  
 -0.11428571428571432 
 -0.06349206349206349 
 -0.04040404040404039 
 -0.02797202797202797 
 -0.020512820512820523
 -0.13333333333333333 

In [145]:
M = Tridiagonal(ld, d, ud)
Base.print_matrix(IOContext(stdout, :limit => false), M)

 -0.6666666666666666   1.0                   ⋅                     ⋅                     ⋅                     ⋅                     ⋅                      ⋅                 
  1.0                 -0.2666666666666666   0.5                    ⋅                     ⋅                     ⋅                     ⋅                      ⋅                 
   ⋅                   0.5                 -0.11428571428571432   0.3333333333333333     ⋅                     ⋅                     ⋅                      ⋅                 
   ⋅                    ⋅                   0.3333333333333333   -0.06349206349206349   0.25                   ⋅                     ⋅                      ⋅                 
   ⋅                    ⋅                    ⋅                    0.25                 -0.04040404040404039   0.2                    ⋅                      ⋅                 
   ⋅                    ⋅                    ⋅                     ⋅                    0.2                  -0.0279720279720

In [99]:
R = rand(MersenneTwister(0), Nz+1)
R[1] = 0
R[Nz+1] = 0
R[2:Nz] = R[2:Nz] .- mean(R[2:Nz])
R

9-element Array{Float64,1}:
  0.0                 
  0.646759570592553   
 -0.09903116920019828 
 -0.08626812086761892 
  0.015283141997317506
 -0.06012040929196083 
 -0.22129530140185383 
 -0.19532771182823871 
  0.0                 

In [100]:
g = [0, ΔzC..., 0] .* R

9-element Array{Float64,1}:
  0.0                
  0.9701393558888294 
 -0.2475779230004957 
 -0.30193842303666624
  0.06877413898792878
 -0.33066225110578457
 -1.4384194591120498 
 -1.4649578387117903 
  0.0                

In [101]:
M \ g

9-element Array{Float64,1}:
   2.3002238536898085
   2.3002238536898085
  -0.3599451419121496
  -4.373042120492275 
  -2.1855075430945563
   5.263796459781347 
   1.6913948370350311
 -15.928132944023416 
 -15.928132944023416 

In [112]:
ϕ = tridiag(M, g)

9-element Array{Float64,1}:
   2.300223853689816 
   2.300223853689816 
  -0.3599451419121573
  -4.373042120492291 
  -2.185507543094551 
   5.263796459781368 
   1.691394837035028 
 -15.928132944023446 
 -15.928132944023416 

In [103]:
∇²ϕ = zeros(Nz+1)

9-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [113]:
for k in 2:Nz-1
    ∇²ϕ[k] = ((ϕ[k-1] - ϕ[k]) / ΔzF[k-1]) - ((ϕ[k] - ϕ[k+1]) / ΔzF[k]) / ΔzC[k]
end
∇²ϕ

9-element Array{Float64,1}:
  0.0                
 -0.5320337991203947 
  0.9478847855552597 
  1.4592286916043633 
 -0.27599986242667424
 -1.5814608421840641 
  0.259790217485181  
  0.0                
  0.0                

In [114]:
R

9-element Array{Float64,1}:
  0.0                 
  0.646759570592553   
 -0.09903116920019828 
 -0.08626812086761892 
  0.015283141997317506
 -0.06012040929196083 
 -0.22129530140185383 
 -0.19532771182823871 
  0.0                 